# Analysing info about driver's licenses

Can the number of newly granted driving licenses differ significantly in one year from the previous one? Who gets a driving license more often, men or women? What about their age? These questions and more are a subject of this little project. 

## Key questions about the data

## About the data

Datasets come from dane.gov.pl website. They contain info about newly granted driver licenses in Poland. There are four separate datasets - they contain info about January and February for both 2024 and 2023. I used only these two months because at the time of writing, newer data is not available. However the analysis could be fairly easy widened into more months using older datasets.  <br>
Let's take a quick look at the data

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import os


In [65]:
csv_files_22 = [f for f in os.listdir('2022') if f.endswith('.csv')] #all csv files in folder 
csv_files_23 = [f for f in os.listdir('2023') if f.endswith('.csv')]
dfs_2022 = [] #list
dfs_2023 = []

for file in csv_files_22:
    file_path = os.path.join('2022', file)  # Pełna ścieżka do pliku
    df = pd.read_csv(file_path, encoding='utf-8')  # Wczytanie pliku
    dfs_2022.append(df)

dfs_2022

[     DATA_MC|KOD_WOJ|WOJEWODZTWO|PLEC|WIEK|LICZBA
 0             2022-01|02|WOJ. DOLNOŚLĄSKIE|K|54|4
 1             2022-01|02|WOJ. DOLNOŚLĄSKIE|K|47|5
 2             2022-01|02|WOJ. DOLNOŚLĄSKIE|K|42|5
 3            2022-01|02|WOJ. DOLNOŚLĄSKIE|K|20|89
 4            2022-01|02|WOJ. DOLNOŚLĄSKIE|K|29|22
 ...                                           ...
 1265    2022-01|32|WOJ. ZACHODNIOPOMORSKIE|M|42|1
 1266   2022-01|32|WOJ. ZACHODNIOPOMORSKIE|M|20|52
 1267    2022-01|32|WOJ. ZACHODNIOPOMORSKIE|M|31|8
 1268   2022-01|32|WOJ. ZACHODNIOPOMORSKIE|M|28|12
 1269    2022-01|32|WOJ. ZACHODNIOPOMORSKIE|M|30|4
 
 [1270 rows x 1 columns],
      DATA_MC|KOD_WOJ|WOJEWODZTWO|PLEC|WIEK|LICZBA
 0             2022-02|02|WOJ. DOLNOŚLĄSKIE|K|44|6
 1             2022-02|02|WOJ. DOLNOŚLĄSKIE|K|16|1
 2            2022-02|02|WOJ. DOLNOŚLĄSKIE|K|23|26
 3            2022-02|02|WOJ. DOLNOŚLĄSKIE|K|26|30
 4            2022-02|02|WOJ. DOLNOŚLĄSKIE|K|33|23
 ...                                           ...
 12

In [66]:
for file in csv_files_23:
    file_path = os.path.join('2023', file)  # Pełna ścieżka do pliku
    df = pd.read_csv(file_path, encoding='utf-8')  # Wczytanie pliku
    dfs_2023.append(df)

dfs_2023

[     DATA_MC|KOD_WOJ|WOJEWODZTWO|PLEC|WIEK|LICZBA
 0            2023-01|02|WOJ. DOLNOŚLĄSKIE|K|21|45
 1             2023-01|02|WOJ. DOLNOŚLĄSKIE|K|63|1
 2             2023-01|02|WOJ. DOLNOŚLĄSKIE|K|49|5
 3             2023-01|02|WOJ. DOLNOŚLĄSKIE|K|41|4
 4             2023-01|02|WOJ. DOLNOŚLĄSKIE|K|56|1
 ...                                           ...
 1253    2023-01|32|WOJ. ZACHODNIOPOMORSKIE|M|17|1
 1254   2023-01|32|WOJ. ZACHODNIOPOMORSKIE|M|22|19
 1255    2023-01|32|WOJ. ZACHODNIOPOMORSKIE|M|39|8
 1256    2023-01|32|WOJ. ZACHODNIOPOMORSKIE|M|25|8
 1257   2023-01|32|WOJ. ZACHODNIOPOMORSKIE|M|21|20
 
 [1258 rows x 1 columns],
      DATA_MC|KOD_WOJ|WOJEWODZTWO|PLEC|WIEK|LICZBA
 0             2023-02|02|WOJ. DOLNOŚLĄSKIE|K|48|5
 1            2023-02|02|WOJ. DOLNOŚLĄSKIE|K|23|17
 2            2023-02|02|WOJ. DOLNOŚLĄSKIE|K|36|16
 3           2023-02|02|WOJ. DOLNOŚLĄSKIE|K|19|319
 4            2023-02|02|WOJ. DOLNOŚLĄSKIE|K|33|18
 ...                                           ...
 12